## Plot airborne data vs ground based observations

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [ ]:
%load_ext autoreload
%aimport aesop
%aimport run_smrt_with_realistic_atmosphere
%autoreload 1

Load pits:

In [ ]:
pits = aesop.get_pit_topography()

Load ground data corrected for the atmosphere:

In [ ]:
sbr = pd.read_csv('../Data/ground_obs/more89forMel.csv')

In [ ]:
tb89v0 = run_smrt_with_realistic_atmosphere.ground_based_data_corrected(sbr, '89V0', pits)

Load airborne observations of the relevant surface type for each pit:

In [ ]:
obs_c087 = aesop.get_obs_per_aoi('C087')
obs_c090 = aesop.get_obs_per_aoi('C090')

In [ ]:
channel = 'M16-89'
tb_obs = []
for p in pits.index:
    topo = pits.loc[p].topo_type
    if p == 'MetS':
        p = 'A04'
    pit_obs_c087, pit_obs_c090 = obs_c087.loc[p[:3]][channel], obs_c090.loc[p[:3]][channel]
    tb_obs.append(np.concatenate([(pit_obs_c087[pit_obs_c087.topo_type==topo].TB),
                                  (pit_obs_c090[pit_obs_c090.topo_type==topo].TB)]))

Get median, errmin and errmax for airborne observations:

In [ ]:
def extract_box_plot_info(tb):
    percentile = []
    for p, data in enumerate(tb):
        qu1, qu2, qu3 = np.percentile(data,q=[25,50,75])
        errmin = qu2 - qu1
        errmax = qu3 - qu2
        percentile.append([errmin, qu2, errmax])
    return np.asarray(percentile)

In [ ]:
airborne_89v0 = extract_box_plot_info(tb_obs)

Plot airborne vs ground based observations at nadir:

In [ ]:
plt.close()
fig, ax = plt.subplots(figsize=(6,6))

ax.errorbar(tb89v0[:,0], airborne_89v0[:,1],
             xerr=[tb89v0[:,1],tb89v0[:,2]], yerr=[airborne_89v0[:,0],airborne_89v0[:,2]],
             fmt='o', label='89V0', color='b', alpha=0.6)

axins = zoomed_inset_axes(ax, 2, loc=2) # zoom = 6
axins.plot(tb89v0[:,0], airborne_89v0[:,1], 'bo', alpha=0.6)
axins.set_xlim(175, 205) # Limit the region for zoom
axins.set_ylim(175, 187)

axins.set_xticks([])  # Not present ticks
axins.set_yticks([])
#
## draw a bbox of the region of the inset axes in the parent axes and
## connecting lines between the bbox and the inset axes area
mark_inset(ax, axins, loc1=3, loc2=4, fc="none", ec="0.5", alpha=0.2)    
    
    
for label, x, y in zip(pits.index, tb89v0[:,0], airborne_89v0[:,1]):
    if label in ['A05W', 'A03W']:
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
    else:
        axins.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
    
ax.plot(np.arange(140, 260, 10), np.arange(140, 260, 10), 'k--', alpha=0.2)
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(160, 250)
ax.set_ylim(160, 250)
ax.tick_params(axis='x')
ax.tick_params(axis='y')
#plt.legend()
#plt.title('Flight vs SBR @89GHz')
ax.set_xlabel('Ground Observed TB [K]')
ax.set_ylabel('Airborne Observed TB [K]')
fig.canvas.width = '600px'
fig.canvas.height = '600px'

In [ ]:
from scipy.stats import linregress

In [ ]:
x = tb89v0[:,0]
y = airborne_89v0[:,1]
mask = ~np.isnan(x)

slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
print ('r^2: ',r_value**2)
print ('ME: ', np.sum(y[mask]-x[mask]) / len(y[mask]))
print ('rmse: ', np.sqrt(np.sum((y[mask]-x[mask])**2) / len(y[mask])))

### Plot airborne vs ground based data at 50/55 deg:

At nadir using observations within the relevant AOI for the surface type of each pit. 

For most runs instruments remained at nadir over the AOIS and so for observations at 50deg take obs within 250m of the pits - most of this data is actually from C090 run 3 where instruments were forward viewing (50deg) over the AOIs.

Load airborne data at 50deg:

In [ ]:
c087_marss = '../Data/raw_airborne/metoffice-marss_faam_20180316_r001_c087.nc'
c090_marss = '../Data/raw_airborne/metoffice-marss_faam_20180320_r001_c090.nc'
datafiles = [c087_marss, c090_marss]

In [ ]:
def subset_data(datafiles, theta):
    for f, file in enumerate(datafiles):
        data = xr.open_dataset(file)
        if f == 0:
            subset = data.where((data.sensor_view_angle > theta-5) & 
                                    (data.sensor_view_angle<theta+5), drop=True)
        else:
            subset = xr.concat([subset, data.where((data.sensor_view_angle > theta-5) & 
                                    (data.sensor_view_angle<theta+5), drop=True)], dim='time')
    return subset.dropna(dim='time')

In [ ]:
subset50 = subset_data(datafiles, 50)

Get data within 250m of pits:

In [ ]:
pits_aesop = aesop.get_pits()
pits_aesop = pits_aesop[~pits_aesop.index.str.contains('A1')]

tb_obs_50 = aesop.find_flights_near_pits(subset50, 0, pits_aesop, 0.25)

Get median, errmin and errmax for airborne observations:

In [ ]:
airborne_89v50 = extract_box_plot_info(tb_obs_50)

Calculate TB for both polarisations at 50 deg:  
H * cos^2 (pol angle) + V * sin^2 (pol angle)

In [ ]:
def extract_flight_data_for_pit(subset, pit, chan):
    TB = []
    for fl in pit:
        TB.append([subset.sel(time=fl, channel=subset.channel[chan]).brightness_temperature.values,
              subset.sel(time=fl).bs_latitude.values, subset.sel(time=fl).bs_longitude.values,
                  subset.sel(time=fl, channel=subset.channel[chan]).detected_polarisation])
    return np.vstack(TB)

def get_pol_angle(subset, channel):
    pits = pits_aesop
    nearpits = []
    for p in range(len(pits)):
        pit_distance = aesop.distance(subset.bs_latitude.astype('float64'), subset.bs_longitude.astype('float64'), pits.lat.values[p], pits.long.values[p])
        nearpits.append(pit_distance[pit_distance<0.25].time.values)
    
    pol_angle = []
    for pit in nearpits:
        try:

            pol_angle.append(extract_flight_data_for_pit(subset, pit, channel)[:,3])
        except:
            pol_angle.append(np.nan)
    
    return pol_angle

In [ ]:
pol_angle = get_pol_angle(subset50, channel=0)

In [ ]:
tb89v55 = run_smrt_with_realistic_atmosphere.ground_based_data_corrected(sbr, '89V55', pits)
tb89h55 = run_smrt_with_realistic_atmosphere.ground_based_data_corrected(sbr, '89H55', pits)

In [ ]:
# Median polarisation angle by pit
pol = extract_box_plot_info(pol_angle)[:,1]
# 0th value in tb89h55 array is mean
tb55 = tb89h55[:,0] * (np.cos(np.radians(pol)))**2 + tb89v55[:,0] * (np.sin(np.radians(pol)))**2

Plot airborne vs ground based observations at nadir and 50/55 deg:

In [ ]:
fig = plt.figure(figsize=(8,6))

plt.errorbar(tb89v0[:,0], airborne_89v0[:,1],
             xerr=[tb89v0[:,1],tb89v0[:,2]], yerr=[airborne_89v0[:,0],airborne_89v0[:,2]],
             fmt='o', label='89V0', color='b', alpha=0.6)

plt.errorbar(tb55, airborne_89v50[:,1],
             xerr=[tb89v55[:,1],tb89v55[:,2]], yerr=[airborne_89v50[:,0],airborne_89v50[:,2]],
             fmt='o', label='89V55', color='k', alpha=0.6)

plt.plot(np.arange(140, 260, 10), np.arange(140, 260, 10), 'k--', alpha=0.2)
plt.xlim(150, 250)
plt.ylim(150, 250)
plt.legend()
plt.title('Flight vs SBR @89GHz')
plt.xlabel('Ground Observed TB [K]')
plt.ylabel('Airborne Observed TB [K]')